In [2]:
!pip install mysql-connector-python
import mysql.connector as mysql
import haversine as hs
from tabulate import tabulate


In [3]:

try:
    connection = mysql.connect(host="localhost", database="strava_db", user="root", password="sveinung", port=3306)
    cursor = connection.cursor()
except Exception as e:
    print("ERROR: Failed to connect to db:", e)


## 1.
### How many users, activities and trackpoints are there in the dataset (after it is inserted into the database).


In [52]:
cursor.execute("""
SELECT 
    (SELECT count(id) FROM User) AS user_count,
    (SELECT count(id) FROM Activity) AS activity_count,
    (SELECT count(id) FROM TrackPoint) AS trackpoint_count;
""")
result_1 = cursor.fetchall()
print(tabulate([*result_1], headers=["user_count","activity_count","trackpoint_count"]))

  user_count    activity_count    trackpoint_count
------------  ----------------  ------------------
         182             16030             9636829


## 2.
### Find the average, maximum and minimum number of trackpoints per user.

In [49]:
cursor.execute("""
    SELECT 
        AVG(trackpoints_count) AS average_trackpoints, 
        MIN(trackpoints_count) AS minimum_trackpoints,
        MAX(trackpoints_count) AS maximum_trackpoints 
    FROM (
        SELECT 
            User.id, 
            COUNT(TrackPoint.id) AS trackpoints_count
        FROM User
        INNER JOIN Activity ON User.id = Activity.user_id
        INNER JOIN TrackPoint ON Activity.id = TrackPoint.activity_id
        GROUP BY User.id
    ) AS trackpoints_per_user
""")
result_2 = cursor.fetchall()
print(tabulate([*result_2], headers=["average_trackpoints","minimum_trackpoints","maximum_trackpoints"]))


  average_trackpoints    minimum_trackpoints    maximum_trackpoints
---------------------  ---------------------  ---------------------
              55704.2                     17                1007827


## 3.
### Find the top 15 users with the highest number of activities.


In [47]:
cursor.execute("""
    SELECT 
        User.id, 
        COUNT(DISTINCT Activity.id) AS activity_count
    FROM User
    INNER JOIN Activity ON User.id = Activity.user_id
    GROUP BY User.id
    ORDER BY activity_count DESC
    LIMIT 15;
""")

result_3 = cursor.fetchall()
print(tabulate([*result_3], headers=["user_id","activity_count"]))

  user_id    activity_count
---------  ----------------
      128              2101
      153              1791
      025               715
      163               704
      062               691
      144               563
      041               398
      085               364
      004               346
      140               345
      167               320
      068               278
      017               265
      003               261
      014               232


## 4.
### Find all users who have taken a bus.

In [46]:
cursor.execute("""
    SELECT DISTINCT User.id
    FROM User
    INNER JOIN Activity ON User.id = Activity.user_id
    WHERE Activity.transportation_mode = 'bus';
""")
result_4 = cursor.fetchall()
print(tabulate([*result_4], headers=["user_id"]))

  used_id
---------
      010
      052
      062
      073
      081
      084
      085
      091
      092
      112
      125
      128
      175


## 5. 
### List the top 10 users by their amount of different transportation modes.

In [45]:
cursor.execute("""
    SELECT 
        User.id, 
        COUNT(DISTINCT Activity.transportation_mode) AS mode_count
    FROM User
    INNER JOIN Activity ON User.id = Activity.user_id
    WHERE Activity.transportation_mode IS NOT NULL
    GROUP BY User.id
    ORDER BY mode_count DESC
    LIMIT 10;
""")
result_5 = cursor.fetchall()
print(tabulate([*result_5], headers=["user_id","transportation_mode_count"]))

  used_id    transportation_mode_count
---------  ---------------------------
      128                            9
      062                            7
      085                            4
      084                            3
      163                            3
      058                            3
      081                            3
      078                            3
      112                            3
      020                            2


## 6.
### Find activities that are registered multiple times. You should find the query even if it gives zero result.

In [44]:
cursor.execute("""
    SELECT 
        user_id, 
        transportation_mode, 
        start_date_time, 
        end_date_time, 
        COUNT(*) AS occurrence_count
    FROM Activity
    GROUP BY user_id, transportation_mode, start_date_time, end_date_time
    HAVING COUNT(*) > 1;
""")
result_6 = cursor.fetchall()
print(tabulate([*result_6], headers=["user_id","transportation_mode","start_date_time","end_date_date","invalid_activites"]))

user_id    transportation_mode    start_date_time    end_date_date    invalid_activites
---------  ---------------------  -----------------  ---------------  -------------------


## 7. a) 
### Find the number of users that have started an activity in one day and ended the activity the next day.

   Not sure if the task asks for us to include the activities that are not classified, we therefore make the assumption that we should not list the 'NULL' activities.

In [4]:
cursor.execute("""
    SELECT 
        COUNT(DISTINCT user_id) 
    FROM Activity
    WHERE DATE(start_date_time) != DATE(end_date_time);
""")

result_7a = cursor.fetchall()
print(result_7a[0][0], "people started an activity and ended it on another date")

98 people started an activity and ended it on another date


## b) 
### List the transportation mode, user id and duration for these activities.

Same assumption

In [62]:
cursor.execute("""
    SELECT 
        user_id, 
        transportation_mode, 
        TIMESTAMPDIFF(MINUTE, start_date_time, end_date_time) AS duration_minutes
    FROM Activity
    WHERE DATE(start_date_time) != DATE(end_date_time) AND transportation_mode IS NOT NULL;
""")
result_7b = cursor.fetchall()
print(tabulate([*result_7b], headers=["user_id","transportation_mode","actity_duration"]))


  user_id  transportation_mode      actity_duration
---------  ---------------------  -----------------
      020  bike                                 610
      021  walk                                 237
      058  car                                   36
      062  walk                                  83
      085  bus                                   58
      115  car                                   88
      115  car                                   62
      115  car                                   60
      115  car                                   65
      115  car                                   83
      115  car                                   54
      115  car                                   68
      115  car                                   69
      115  car                                   63
      115  car                                   61
      115  car                                   55
      115  car                                   84
      115  c

## 8.
### Find the number of users which have been close to each other in time and space. Close is defined as the same space (50 meters) and for the same half minute (30 seconds)


In [16]:
cursor.execute("""
SELECT user.id, date, lat,lon FROM Trackpoint
INNER JOIN Activity
ON Activity.id = Trackpoint.activity_id
INNER JOIN user
on user.id = activity.user_id
ORDER BY date ASC

""")
sorted_trackpoints = cursor.fetchall()

In [19]:
meetups = []
for i in range(len(sorted_trackpoints)):
    user1 = sorted_trackpoints[i]
    user1_id = user1[0]
    user1_date = user1[1]
    user1_lat = user1[2]
    user1_lon = user1[3]
    for j in range(i+1,len(sorted_trackpoints)):
        user2 = sorted_trackpoints[j]
        user2_id = user2[0]
        if user1_id == user2_id: continue
        user2_date = user2[1]
        seconds_difference = (user2_date - user1_date).total_seconds()
        if seconds_difference > 30: break
        user2_lat = user2[2]
        user2_lon = user2[3]
        distance = hs.haversine((user1_lat,user1_lon),(user2_lat,user2_lon))
        if distance > 0.05: continue
        meetups.append((user1_id,user2_id))
        

In [26]:
dic = {}
for tup in meetups:
    id1 = tup[0]
    id2 = tup[1]
    if id1 not in dic: dic[id1] = {id2}
    else: dic[id1].add(id2)
    if id2 not in dic: dic[id2] = {id1}
    else: dic[id2].add(id1)

print(len(dic), "users have been close to each other at some point")

number of users who have meet eachother: 121


## 9.
### Find the top 15 users who have gained the most altitude meters.

In [39]:
cursor.execute("""
SELECT
    u.id AS user_id,
    SUM(CASE WHEN tp1.altitude > tp2.altitude THEN tp1.altitude - tp2.altitude ELSE 0 END) AS total_altitude_gain
FROM
    User AS u
JOIN
    Activity AS a ON u.id = a.user_id
JOIN
    TrackPoint AS tp1 ON a.id = tp1.activity_id
JOIN
    TrackPoint AS tp2 ON a.id = tp2.activity_id AND tp1.id = tp2.id - 1
WHERE
    tp1.altitude IS NOT NULL AND tp2.altitude IS NOT NULL
GROUP BY
    u.id
ORDER BY
    total_altitude_gain DESC
LIMIT
    15;

""")


result_9 = cursor.fetchall()
print(tabulate([*result_9], headers=["user_id","altitude_gain"]))

  user_id     altitude_gain
---------  ----------------
      153       2.06276e+06
      128       1.88582e+06
      004       1.14473e+06
      003  793859
      163  770091
      085  753055
      062  670128
      041  655190
      144  568877
      030  563027
      039  511261
      025  470196
      084  453032
      167  417595
      000  413662


## 10.
### Find the users that have traveled the longest total distance in one day for each transportation mode.

In [63]:
cursor.execute("""

WITH distances as 
(
    SELECT 
        a.user_id,
        a.transportation_mode,
        SUM(
            6371 * ACOS(
                COS(RADIANS(tp1.lat)) * COS(RADIANS(tp2.lat)) * COS(RADIANS(tp2.lon) - RADIANS(tp1.lon)) +
                SIN(RADIANS(tp1.lat)) * SIN(RADIANS(tp2.lat))
            )
        ) as total_distance,
        DATE(tp1.date) as travel_date
    FROM 
        Activity a
    JOIN 
        TrackPoint tp1 ON a.id = tp1.activity_id
    JOIN 
        TrackPoint tp2 ON a.id = tp2.activity_id AND tp1.id = tp2.id - 1
    WHERE 
        a.transportation_mode IS NOT NULL
    GROUP BY 
        a.user_id, a.transportation_mode, travel_date
), ActivityMaxDistance AS (
    SELECT 
        d.transportation_mode,
        MAX(d.total_distance) AS max_distance 
    FROM distances d
    GROUP BY d.transportation_mode
)

SELECT user_id, d.transportation_mode, max_distance
FROM distances d
JOIN ActivityMaxDistance amd ON d.transportation_mode = amd.transportation_mode AND d.total_distance = amd.max_distance
ORDER BY d.transportation_mode DESC;

""")

result_10 = cursor.fetchall()
print(tabulate([*result_10], headers=["user_id", "transportation_mode", "max_distance"]))


  user_id  transportation_mode      max_distance
---------  ---------------------  --------------
      108  walk                       22.8074
      139  walk                       22.8074
      062  train                     277.257
      128  taxi                       40.2232
      128  subway                     33.9351
      062  run                         0.0332528
      128  car                       398.173
      128  bus                       207.413
      128  boat                       65.5547
      128  bike                       63.113
      128  airplane                 2527.12


## 11.
### Find all users who have invalid activities, and the number of invalid activities per user

In [54]:
cursor.execute("""
WITH InvalidActivities AS (
    SELECT
        tp1.activity_id AS a_id
    FROM
        TrackPoint AS tp1
    JOIN TrackPoint AS tp2 ON tp1.id + 1 = tp2.id
    WHERE
        tp1.activity_id = tp2.activity_id
        AND TIMESTAMPDIFF(MINUTE, tp1.date, tp2.date) >= 5
    GROUP BY tp1.activity_id
)
SELECT
    a.user_id,
    COUNT(ia.a_id) AS invalid_activity_count
FROM
    Activity AS a
JOIN InvalidActivities AS ia ON a.id = ia.a_id
GROUP BY a.user_id;


""")

result_11 = cursor.fetchall()
print(tabulate([*result_11], headers=["user_id","invalid_activity_count"]))

  user_id    invalid_activity_count
---------  ------------------------
      000                       101
      001                        45
      002                        98
      003                       179
      004                       219
      005                        45
      006                        17
      007                        30
      008                        15
      009                        31
      010                        50
      011                        32
      012                        43
      013                        29
      014                       114
      015                        46
      016                        20
      017                       129
      018                        27
      019                        31
      020                        20
      021                         7
      022                        55
      023                        11
      024                        27
      025                   

## 12.
### Find all users who have registered transportation_mode and their most used transportation_mode.


In [55]:
cursor.execute("""
SELECT 
    user_id, 
    transportation_mode
FROM 
    (SELECT 
        user_id, 
        transportation_mode, 
        RANK() OVER(PARTITION BY user_id ORDER BY COUNT(*) DESC) as rnk
     FROM Activity
     WHERE transportation_mode IS NOT NULL
     GROUP BY user_id, transportation_mode) AS Ranked
WHERE 
    rnk = 1
ORDER BY 
    user_id ASC;

""")

result_12 = cursor.fetchall()
print(tabulate([*result_12], headers=["user_id","most_used_transportation_mode"]))

  user_id  most_used_transportation_mode
---------  -------------------------------
      010  taxi
      020  bike
      021  walk
      052  bus
      056  bike
      058  taxi
      058  car
      058  walk
      060  walk
      062  walk
      062  bus
      064  bike
      065  bike
      067  walk
      069  bike
      073  walk
      075  walk
      076  car
      078  walk
      080  taxi
      080  bike
      081  bike
      082  walk
      084  walk
      085  walk
      086  car
      087  walk
      089  car
      091  bus
      091  walk
      092  bus
      092  walk
      097  bike
      098  taxi
      101  car
      102  bike
      107  walk
      108  walk
      111  taxi
      112  walk
      115  car
      117  walk
      125  bike
      126  bike
      128  car
      136  walk
      138  bike
      139  bike
      144  walk
      153  walk
      161  walk
      163  bike
      167  bike
      175  bus
